In [2]:
import pandas as pd
from pymystem3 import Mystem
import spacy
nlp = spacy.load("ru_core_news_lg")

### Data Load

In [3]:
df_manual = []
with open("../edz_dataset.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.replace("\ufeff" ,"")
        line = line.replace("\n" ,"")
        df_manual.append([line[:-1], 1])
        
with open("../edz_neg_dataset.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.replace("\ufeff" ,"")
        line = line.replace("\n" ,"")
        df_manual.append([line[:-1], 0])
df_manual = pd.DataFrame(df_manual, columns=["text", "label"])


In [4]:
df_manual

,text,label
0,завтра буду учить английский весь ден,1
1,Завтра я пойду на работ,1
2,послезавтра мы планируем ехать на дач,1
3,к вечеру я планирую закончить отче,1
4,в воскресенье после обеда мы пойдем в рестора,1
...,...,...
438,Сенат США потребовал прекратить военную помощь...,0
439,СБУ забила тревогу из-за российских «Паруса» и...,0
440,Владельцы Skoda смогут бесплатно парковаться в...,0
441,Европа отказалась запускать спутники с Восточног,0


подмешаем немного фраз из заголвков лентыб думаю что планов там будет очень малая часть

In [5]:
%scrpit false
df = pd.read_csv("/Users/R1303-M-4-Stud/PlanningLooker/lenta-ru-news_csv/lenta-ru-news.csv", nrows=200)
add_rows = []
for i in df["title"]:
    add_rows.append(i)
with open("/Users/R1303-M-4-Stud/PlanningLooker/edz_neg_dataset.txt", "a") as f:
    for i in add_rows:
        f.write(i+"\n")

UsageError: Line magic function `%scrpit` not found.


### Tags review

#### SpaCy

In [6]:
tags = []
doc = nlp(df_manual.loc[22, "text"])
for token in doc:
    tags.append([token.text, token.lemma_, token.ent_type_, token.pos_, token.tag_, token.dep_, token.head])
postok = {str(i): list(doc).index(i) for i in doc}
pd.DataFrame(tags, columns = ["Token", "lemma", "entity", "full", "entity", "dep type", "head"])

,Token,lemma,entity,full,entity,dep type,head
0,к,к,,ADP,ADP,case,пяти
1,пяти,пять,,NUM,NUM,nummod,утра
2,утра,утро,,NOUN,NOUN,obl,завершится
3,хакатон,хакатон,,PROPN,PROPN,nsubj,завершится
4,завершится,завершиться,,VERB,VERB,ROOT,завершится
5,и,и,,CCONJ,CCONJ,cc,сдадим
6,мы,мы,,PRON,PRON,nsubj,сдадим
7,сдадим,сдать,,VERB,VERB,conj,завершится
8,результат,результат,,NOUN,NOUN,obj,сдадим


#### MyStem

In [7]:
stem = Mystem()
stem.analyze(df_manual.loc[22, "text"])

[{'analysis': [{'lex': 'к', 'wt': 0.9999551798, 'gr': 'PR='}], 'text': 'к'},
 {'text': ' '},
 {'analysis': [{'lex': 'пять', 'wt': 1, 'gr': 'NUM=(пр|дат|род)'}],
  'text': 'пяти'},
 {'text': ' '},
 {'analysis': [{'lex': 'утро',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)'}],
  'text': 'утра'},
 {'text': ' '},
 {'analysis': [{'lex': 'хакатон',
    'wt': 0.2835818696,
    'qual': 'bastard',
    'gr': 'S,муж,неод=(вин,ед|им,ед)'}],
  'text': 'хакатон'},
 {'text': ' '},
 {'analysis': [{'lex': 'завершаться',
    'wt': 1,
    'gr': 'V,нп=непрош,ед,изъяв,3-л,сов'}],
  'text': 'завершится'},
 {'text': ' '},
 {'analysis': [{'lex': 'и', 'wt': 0.9999770357, 'gr': 'CONJ='}], 'text': 'и'},
 {'text': ' '},
 {'analysis': [{'lex': 'мы', 'wt': 1, 'gr': 'SPRO,мн,1-л=им'}], 'text': 'мы'},
 {'text': ' '},
 {'analysis': [{'lex': 'сдавать',
    'wt': 1,
    'gr': 'V,пе=(мн,пов,1-л,сов|непрош,мн,изъяв,1-л,сов)'}],
  'text': 'сдадим'},
 {'text': ' '},
 {'analysis': [{'lex': 'результат',
    'wt':

#### Natasha

In [8]:
import natasha

In [9]:
%script false
# не работает на новом питоне
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
text = df.loc[23, "text"]
doc = Doc(text)

UsageError: Line magic function `%script` not found (But cell magic `%%script` exists, did you mean that instead?).


### BoW

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [11]:
model_bow = CountVectorizer()
X = model_bow.fit_transform(df_manual.text).toarray()
y = df_manual.label
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

Experiment 1 F1 - 0.89

In [13]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
print("Initial experiment")
print(classification_report(y_test, y_pred))

Initial experiment
              precision    recall  f1-score   support

           0       0.84      0.98      0.90        58
           1       0.98      0.79      0.88        53

    accuracy                           0.89       111
   macro avg       0.91      0.89      0.89       111
weighted avg       0.90      0.89      0.89       111



In [14]:
model.predict(model_bow.transform(["Я покушал колбаску с сыром"]))

array([0])

In [15]:
model.predict(model_bow.transform(["В эту пятницу у меня занятия танцами "]))

array([1])

### Extract lemmas - F1 0.94

In [38]:
import string
from nltk.corpus import stopwords
def preprocess(text):
    lemmas = []
    doc = nlp(text)
    for tok in doc:
        lemmas.append(tok.lemma_)
    eng_stopwords = ["и", "а"]
    punct = string.punctuation + ","
    output = [token for token in lemmas if token not in eng_stopwords]
    output = [token for token in lemmas if token.strip() not in punct]
    return " ".join(output)
preprocess("Слушай сюда! Завтра мы с мамой идем пасху святить, с нами пойдешь")

'слушать сюда завтра мы с мама идти пасха святить с нами пойти'

for i in 

In [27]:
df_manual["lemmas"] = df_manual["text"].apply(lambda x: preprocess(x))
model_bow = CountVectorizer()
X = model_bow.fit_transform(df_manual.lemmas).toarray()
y = df_manual.label
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True)

In [28]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [29]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
print("Initial experiment")
print(classification_report(y_test, y_pred))

Initial experiment
              precision    recall  f1-score   support

           0       0.91      0.98      0.94        61
           1       0.98      0.88      0.93        50

    accuracy                           0.94       111
   macro avg       0.94      0.93      0.94       111
weighted avg       0.94      0.94      0.94       111



И тут кстати я наткнулся на проблему с тем что модель неадекватно работает когда нет обьекта.

In [32]:
model.predict(model_bow.transform(["завтра вечером собираемся в кино"]))

array([0])

In [33]:
model.predict(model_bow.transform(["Мы завтра вечером собираемся в кино"]))

array([1])

In [37]:
model.predict(model_bow.transform(["Мы собираемся в кино вечером 9 апреля"]))

array([0])